El flujo principal del código realiza una combinación de técnicas para implementar un sistema de recuperación de información híbrido (Hybrid Retrieval System) enfocado en la consulta semántica, el uso de embeddings y el reranking contextual basado en palabras clave y similitud semántica

# 1. Configuración Inicial

In [1]:
import os
import sys
import pprint
import importlib.util

# -----------------------------------------
# 1. Configuración Inicial
# -----------------------------------------

# Agregar el directorio base del proyecto a sys.path
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), "../"))
SRC_PATH = os.path.join(BASE_DIR, "src")

if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)
    print(f"'src' agregado a sys.path correctamente: {SRC_PATH}")

# Intentar importar módulos necesarios
try:
    from vector_store_client import connect_to_qdrant, search_qdrant
    from embedding import generate_embeddings
    import ragas  # Para evaluación posterior

    # Conectar a Qdrant
    qdrant_client = connect_to_qdrant()
    index_name = "regulacion_alimentaria"
    print("Conexión a Qdrant establecida con éxito.")
except ModuleNotFoundError as e:
    print(f"Error al importar módulos: {e}")

'src' agregado a sys.path correctamente: /Users/maximilianocruz/Documents/GitHub/practicos-rag/src


/Users/maximilianocruz/anaconda3/envs/ragas_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Modelo 'all-MiniLM-L6-v2' cargado correctamente.
Cargando el módulo 'embedding'
Conexión a Qdrant establecida con éxito.


- Propósito: Prepara el entorno importando módulos personalizados y configurando la conexión al almacén vectorial.

- Detalles:
    - Agrega el directorio src al sys.path para facilitar las importaciones.
    - Importa y conecta con Qdrant, un vector store que almacena representaciones densas de documentos.
    - Importa herramientas de embedding, búsqueda y evaluación.
    - Mensajes informativos indican el éxito de la configuración.

# 2. Importación Manual de Funciones y Modelos

In [2]:
# -----------------------------------------
# 2. Importación Manual de Funciones y Modelos
# -----------------------------------------

# Cargar el módulo `reranking` manualmente
reranking_path = os.path.join(SRC_PATH, "retrievers", "reranking.py")
spec = importlib.util.spec_from_file_location("retrievers.reranking", reranking_path)
reranking = importlib.util.module_from_spec(spec)
spec.loader.exec_module(reranking)

# Cargar el módulo `hybrid_search` manualmente
hybrid_search_path = os.path.join(SRC_PATH, "retrievers", "hybrid_search.py")
spec = importlib.util.spec_from_file_location("retrievers.hybrid_search", hybrid_search_path)
hybrid_search = importlib.util.module_from_spec(spec)
spec.loader.exec_module(hybrid_search)

# Asignar funciones y modelos a variables
embedding_model = reranking.embedding_model
generate_keywords_with_t5 = reranking.generate_keywords_with_t5
expand_keywords_with_similarity = reranking.expand_keywords_with_similarity
hybrid_reranking = hybrid_search.hybrid_reranking

print("Importaciones manuales de `retrievers` completadas.")

Device set to use mps:0


Importaciones manuales de `retrievers` completadas.


- Propósito: Cargar módulos específicos de forma dinámica desde rutas personalizadas.

- Detalles:
    - Importa reranking y hybrid_search manualmente, que contienen funciones críticas para reranking y búsqueda híbrida.

- Variables clave:
    - embedding_model: Modelo para generar embeddings.
	- generate_keywords_with_t5: Generador de palabras clave basado en T5.
	- expand_keywords_with_similarity: Expansión semántica de palabras clave.
	- hybrid_reranking: Algoritmo híbrido para reranking de resultados.

# 3. Flujo Principal: Búsqueda y Reranking

In [3]:
# -----------------------------------------
# 3. Flujo Principal: Búsqueda y Reranking
# -----------------------------------------

# Definir una consulta
query = "¿Qué requisitos deben cumplir los alimentos para ser etiquetados correctamente en Argentina?"

# Generar embedding para la consulta
query_embedding = generate_embeddings([query])[0]
print("Embedding de la consulta generado con éxito.")

# Realizar búsqueda en Qdrant
limit = 5  # Número de resultados deseados
search_results = search_qdrant(qdrant_client, index_name, query_embedding, limit=limit)
print(f"Resultados de búsqueda para la consulta: '{query}'")

# Mostrar los resultados recuperados
print("\nResultados Recuperados:")
for i, result in enumerate(search_results):
    print(f"\nResultado {i + 1}:")
    pprint.pprint(result)

Generando embeddings para 1 chunks...


Generando embeddings: 100%|██████████| 1/1 [00:00<00:00,  3.48chunk/s]


Total de embeddings generados: 1
Embedding de la consulta generado con éxito.
Resultados de búsqueda para la consulta: '¿Qué requisitos deben cumplir los alimentos para ser etiquetados correctamente en Argentina?'

Resultados Recuperados:

Resultado 1:
ScoredPoint(id='8ca9054a-2576-4472-9011-97b655a83b8e', version=828, score=0.7606783, payload={'content': {'type': 'narrative', 'content': 'Cuando cualquiera de aquellos sea importado, se aplicarán los requeri mientos de este Código; dichas exigencias se considerarán también satisfechas cuando los productos provengan de países que cuenten con niveles de contralor alimentario equiparables a los de la República Argentina a criterio de la Autoridad Sanitaria Nacion al, o cuando utilicen las normas del Codex\nAlimentarius (FAO/OMS).'}}, vector=[-0.035405107, -0.0016469358, -0.02925928, -0.093187645, -0.037644017, 0.039801758, 0.07360328, 0.07232209, 0.0093654385, 0.06479226, 0.09208057, -0.018838344, -0.036653306, -0.0063932994, -0.011714913,

- Propósito: Realizar la búsqueda inicial basada en similitud semántica.

- Detalles:

	1.	Generación de embedding para la consulta:
	    - Usa generate_embeddings para transformar la consulta en una representación vectorial.
	
    2.	Búsqueda en Qdrant:
	    - Realiza una búsqueda basada en embeddings y similitud coseno.
	    - Recupera los documentos más relevantes.
	
    3.	Presentación de resultados:
	    - Muestra los documentos recuperados con sus detalles.

# 4. Reranking con Enfoque Híbrido

In [4]:
# -----------------------------------------
# 4. Reranking con Enfoque Híbrido
# -----------------------------------------

# Generar embedding de la consulta para similitud semántica
query_embedding = embedding_model.encode(query, convert_to_tensor=True)

# Recuperar contenido de los resultados
corpus = [result.payload['content']['content'] for result in search_results]

# Generar palabras clave iniciales
initial_keywords = generate_keywords_with_t5(query)
print(f"Palabras clave iniciales: {initial_keywords}")

# Expandir palabras clave usando similitud semántica
expanded_keywords = expand_keywords_with_similarity(initial_keywords)
print(f"Palabras clave expandidas: {expanded_keywords}")

# Combinar las palabras clave generadas y expandidas
query_keywords = list(set(initial_keywords + expanded_keywords))
print(f"Palabras clave finales: {query_keywords}")

# Aplicar reranking híbrido a los resultados recuperados
reranked_results = hybrid_reranking(search_results, query_embedding, query_keywords)

# Mostrar resultados rerankeados
print("\nResultados Rerankeados:")
for i, result in enumerate(reranked_results):
    print(f"\nResultado {i + 1} (Score: {result.score}):")
    pprint.pprint(result.payload['content']['content'])

Palabras clave iniciales: ['en Argentina? Qué requisitos deben cumplir los alimentos para ser etiquetados correctamente?']
Palabras clave expandidas: ['en Argentina? Qué requisitos deben cumplir los alimentos para ser etiquetados correctamente?']
Palabras clave finales: ['en Argentina? Qué requisitos deben cumplir los alimentos para ser etiquetados correctamente?']

Resultados Rerankeados:

Resultado 1 (Score: 1.5213565914731775):
('Cuando cualquiera de aquellos sea importado, se aplicarán los requeri '
 'mientos de este Código; dichas exigencias se considerarán también '
 'satisfechas cuando los productos provengan de países que cuenten con niveles '
 'de contralor alimentario equiparables a los de la República Argentina a '
 'criterio de la Autoridad Sanitaria Nacion al, o cuando utilicen las normas '
 'del Codex\n'
 'Alimentarius (FAO/OMS).')

Resultado 2 (Score: 1.462089351435291):
('Estas hierbas deberán satisfacer las exigencias establecidas en este código '
 'y/o en la Farmacope

- Propósito: Refinar los resultados iniciales mediante un enfoque híbrido basado en embeddings y palabras clave.

- Detalles:

	1.	Generación de embedding para la consulta:
	    - Genera una representación densa utilizando el modelo de embeddings sentence-transformers (o similar).
	
    2.	Generación de palabras clave con T5:
	    - T5 genera palabras clave relevantes para la consulta.
	
    3.	Expansión de palabras clave:
        - Expande las palabras clave iniciales usando embeddings y similitud semántica.
        - Combina palabras clave iniciales y expandidas en un conjunto final.
	
    4.	Reranking híbrido:
	    - Usa un algoritmo híbrido que combina: Similitud semántica entre embeddings (coseno).
	    - Relevancia contextual ajustada por las palabras clave generadas.
	
    5.	Resultados rerankeados:
	    - Presenta los documentos ajustados con un nuevo puntaje que combina similitud y keywords.

##	Validar los vectores recuperados


In [5]:
for i, result in enumerate(search_results):
    print(f"\nResultado {i + 1}:")
    print(f"ID: {result.id}")
    print(f"Vector: {result.vector}")


Resultado 1:
ID: 8ca9054a-2576-4472-9011-97b655a83b8e
Vector: [-0.035405107, -0.0016469358, -0.02925928, -0.093187645, -0.037644017, 0.039801758, 0.07360328, 0.07232209, 0.0093654385, 0.06479226, 0.09208057, -0.018838344, -0.036653306, -0.0063932994, -0.011714913, 0.0009928783, -0.07211006, 0.0020423546, 0.017155599, 0.022611633, 0.09111109, 0.019141315, -0.096094124, 0.058012314, -0.10772328, 0.012817836, 0.010416923, -0.04039536, -0.008738991, -0.08123934, -0.02944138, 0.082519345, 0.116052285, -0.023211183, 0.035611797, -0.0074750986, 0.08742018, -0.12545072, -0.003362787, -0.025725747, -0.072261214, -0.020794068, -0.083044946, -0.009317257, -0.032159407, -0.05150656, 0.053614605, 0.054939274, -0.03163936, -0.018232027, -0.045386788, -0.06794618, -0.099647045, -0.015954968, 0.014120523, -0.003932136, -0.06938743, -0.066439934, 0.04661802, 0.079683155, 0.025686603, 0.08767364, -0.055047333, 0.004776108, 0.045830965, 0.009956488, 0.057817776, -0.007832612, -0.07122699, 0.040555656, 0

## Revisar el cálculo de similitudes

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Mover el tensor a la CPU y convertirlo en un array de NumPy
query_embedding_np = query_embedding.cpu().numpy()

# Calcular similitudes coseno para cada vector recuperado
for i, result in enumerate(search_results):
    result_vector_np = np.asarray(result.vector)  # Asegurarse de que el vector esté en formato NumPy
    similarity = cosine_similarity([query_embedding_np], [result_vector_np])[0][0]
    print(f"Resultado {i + 1}: Similitud coseno con la consulta: {similarity}")

Resultado 1: Similitud coseno con la consulta: 0.7606782914731776
Resultado 2: Similitud coseno con la consulta: 0.7310447014352909
Resultado 3: Similitud coseno con la consulta: 0.7270330059890295
Resultado 4: Similitud coseno con la consulta: 0.7242912109777143
Resultado 5: Similitud coseno con la consulta: 0.720854509662231


## Verifica los pasos del reranking

In [8]:
for result in reranked_results:
    print(f"ID: {result.id}, Score inicial: {result.score}")

ID: 8ca9054a-2576-4472-9011-97b655a83b8e, Score inicial: 1.5213565914731775
ID: ae7c4377-5cac-46d1-9dcf-1f9a7e8f7853, Score inicial: 1.462089351435291
ID: f2f68e81-18f2-4ea7-be3a-104ccab6e2b0, Score inicial: 1.4540660059890296
ID: d67144f8-1b9d-4c03-9fc7-81db033b0881, Score inicial: 1.4485823609777142
ID: 94a94b00-8b0f-458a-b989-a0c61ee260e3, Score inicial: 1.441709009662231
